# Address node layouting

I have extracted the data from the street directory in tabular form. Next, I will try to add as many coordinates to as many addresses (nodes) as possible and fill in the in-between ones algorithmically.

Recap: I previously tried to achieve extrapolated positions for nodes by using positioning algorithms on the data in NetworkX graph format, but I didn't manage to combine known and computed points in a single layout.

In [1]:
import numpy as np
import pandas as pd

import pickle
import re

import networkx as nx
import matplotlib.pyplot as plt
import plotly.express as px
import geopandas as gpd
from geopy.geocoders import GoogleV3


## The data

In [6]:
#import csv
df_ballymac = pd.read_csv('./data/df_nodes_ballymac_2.csv')
df_ballymac

,Unnamed: 0,index,street,side,node_info,node_type,from,to,second_street,door_number
0,0,62,Abercorn Basin,NaN,Abercorn Basin,NaN,NaN,NaN,NaN,NaN
1,1,63,Abercorn Basin,NaN,"1 & 2. Humphrys, L. J., Agent for Wigan Coal a...",NaN,NaN,NaN,NaN,1.0
2,2,64,Abercorn Basin,NaN,Ulster Iron Works,NaN,NaN,NaN,NaN,NaN
3,3,65,Abercorn Basin,NaN,"MacIlwaine & Lewis, Engineers, Boilermakers an...",NaN,NaN,NaN,NaN,NaN
4,4,66,Abercorn Basin,NaN,Hamilton Dock House,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
3637,3637,38526,Woodstock Street,NaN,"2. Cooke, James, tailor",NaN,NaN,NaN,NaN,2.0
3638,3638,38960,Young's Place,NaN,Off Newtownards Road,off,Newtownards Road,NaN,Newtownards Road,NaN
3639,3639,38961,Young's Place,NaN,Ten small houses,NaN,NaN,NaN,NaN,NaN
3640,3640,38962,Young's Row,NaN,Off Newtownards Road,off,Newtownards Road,NaN,Newtownards Road,NaN


Check what has no door numbers but isn't an intersection: 452 rows - these could be candidates for historical matching.

In [10]:
df_ballymac[df_ballymac['door_number'].isna() & df_ballymac['node_type'].isna()]

,Unnamed: 0,index,street,side,node_info,node_type,from,to,second_street,door_number
0,0,62,Abercorn Basin,NaN,Abercorn Basin,NaN,NaN,NaN,NaN,NaN
2,2,64,Abercorn Basin,NaN,Ulster Iron Works,NaN,NaN,NaN,NaN,NaN
3,3,65,Abercorn Basin,NaN,"MacIlwaine & Lewis, Engineers, Boilermakers an...",NaN,NaN,NaN,NaN,NaN
4,4,66,Abercorn Basin,NaN,Hamilton Dock House,NaN,NaN,NaN,NaN,NaN
5,5,67,Abercorn Basin,NaN,Hamilton Graving Dock,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
3570,3570,38459,Woodstock Road,NaN,"Morrow, Alex., builder",NaN,NaN,NaN,NaN,NaN
3571,3571,38460,Woodstock Road,NaN,Warwick Terrace,NaN,NaN,NaN,NaN,NaN
3625,3625,38514,Woodstock Street,NaN,__________,NaN,NaN,NaN,NaN,NaN
3639,3639,38961,Young's Place,NaN,Ten small houses,NaN,NaN,NaN,NaN,NaN


## Geocoding

### Geocoding intersections (Google API)

In [ ]:
with open('./data/api_key_googlemaps', 'r') as keyfile:
    api_key = keyfile.read()

geolocator = GoogleV3(api_key)

def geocode_intersection(df_row):
    location = geolocator.geocode(df_ballymac['street'] + " & " + df_ballymac['second_street'] + ", Belfast, UK", components={"type":"intersection"})
    if location and 'intersection' in location.raw.get('types', []):
        return (location.latitude, location.longitude)

#add placeholder cols
df_ballymac['latitude'] = df_ballymac['longitude'] = ''

for index, row in df_ballymac.iterrows():
    if 'second_street' in row and not pd.isna(row['second_street']):
        coord = geocode_intersection(row)
        if coord:
            df_ballymac.at[index, 'latitude'], df_ballymac.at[index, 'longitude'] = coord

df_ballymac[df_ballymac['latitude'].notna()]

### Geocoding landmarks (Google API)

### Geocoding landmarks (historical dataset matching)

## Examine data

How many lines (streets with address nodes) do I have?

Of these lines, how mnay have at least two coordinates?

## Interpolate missing coordinates

### Test case: single line (street)
To test the implementation of an algorithm to intepolate coordinates between two known coordinates, I have chosen the street with the most known coordinates.